In [ ]:
import numpy as np
import onnx
from onnx import numpy_helper
import inspect

In [270]:
# Define a dictionary

def MatMul(input, initializer):
    return np.dot(input, initializer)

def Mul(input, initializer): 
    return np.dot(input, initializer)

def Add(input, initializer):
    return input + initializer

def Transpose(input, perm): 
    return np.transpose(input, perm)
    
def Conv(x, w, b, dilations, group, kernel_shape, strides, pads = (0, 0, 0, 0)): 
    batch_size, in_channels, in_height, in_width = x.shape
    out_channels, _, kernel_height, kernel_width = w.shape
    stride_height, stride_width = strides
    pad_height_begin, pad_width_begin, pad_height_end, pad_width_end = pads

    out_height = (in_height + pad_height_begin + pad_height_end - kernel_height) // stride_height + 1
    out_width = (in_width + pad_width_begin + pad_width_end - kernel_width) // stride_width + 1

    # y = np.zeros((batch_size, out_channels, out_height, out_width), dtype=np.float32)
    y = np.zeros((batch_size, out_channels, out_height, out_width), dtype=np.float32)

    x_padded = np.pad(x, ((0, 0), (0, 0), (pad_height_begin, pad_height_end), (pad_width_begin, pad_width_end)), mode='constant')
    
    for i in range(out_height):
        for j in range(out_width):
            h_start = i * stride_height
            h_end = h_start + kernel_height
            w_start = j * stride_width
            w_end = w_start + kernel_width
            x_slice = x_padded[:, :, h_start:h_end, w_start:w_end]
            for k in range(out_channels):
                y[:, k, i, j] = np.sum(x_slice * w[k, :, :, :], axis=(1, 2, 3))
                
    if b is not None:
        y += b.reshape(1, -1, 1, 1)
        
    return y

def Relu(input): 
    return np.maximum(0, input)

def MaxPool(x, kernel_shape, strides, pads = [0, 0, 0, 0]): 
    N, C, H, W = x.shape
    kH, kW = kernel_shape
    sH, sW = strides
    pad_top, pad_bottom, pad_left, pad_right = pads

    # Apply padding to the input array
    x_padded = np.pad(x, ((0, 0), (0, 0), (pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')

    # Calculate the dimensions of the output array
    out_H = (H + pad_top + pad_bottom - kH) // sH + 1
    out_W = (W + pad_left + pad_right - kW) // sW + 1

    # Initialize the output array
    out = np.zeros((N, C, out_H, out_W))

    # Perform max pooling
    for n in range(N):
        for c in range(C):
            for i in range(out_H):
                for j in range(out_W):
                    h_start = i * sH
                    h_end = h_start + kH
                    w_start = j * sW
                    w_end = w_start + kW
                    out[n, c, i, j] = np.max(x_padded[n, c, h_start:h_end, w_start:w_end])

    return out

def Reshape(input, shape): 
    return input[0].reshape(shape)

def Gemm(A, B, C): 
    return np.dot(A, B) + C

def Softmax(x, axis=-1):
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / e_x.sum(axis=axis, keepdims=True)

function_dict = {
    "MatMul": MatMul,
    "Mul": Mul, 
    "Add": Add, 
    "Transpose": Transpose, 
    "Conv": Conv, 
    "Relu": Relu, 
    "MaxPool": MaxPool, 
    "Reshape": Reshape, 
    "Gemm": Gemm, 
    "Softmax": Softmax
    }

In [99]:
def onnx_run(model, input):
    operators = []
    inputs = []
    initializers = []

    # get onnx operators
    operators = [node.op_type for node in model.graph.node]


    # get onnx initializers
    inputs = [i.input[1:] if len(i.input) > 1 else ['None'] for i in model.graph.node]

    # Create a dictionary to map initializer names to their array representations
    init_dict = {inits.name: numpy_helper.to_array(inits) for inits in model.graph.initializer}

    # Iterate over each list in inputs and map names to their initializers
    for inputss in inputs:
        initials = [init_dict[i] for i in inputss if i in init_dict]
        initializers.append(initials)


    # run operators
    result = input
    
    for operator, args, i in zip(operators, initializers, range(len(model.graph.node))): 

        # get onnx attributes

        # Get the function's parameter names
        attr = {attr.name: attr.ints for attr in model.graph.node[i].attribute}
        
        # Filter the params dictionary to only include the parameters that the function accepts
        func_params = inspect.signature(function_dict[operator]).parameters
        filtered_attrs = {k: v for k, v in attr.items() if k in func_params}

        result = function_dict[operator](result, *args, **filtered_attrs)
    
    return result

In [ ]:
onnx_run(model, input)